<img src="http://www.cidaen.es/assets/img/mCIDaeNnb.png" alt="Logo CiDAEN" align="right">

<h1><font size=4>Trabajo Fin de Master (TFM)</font></h1>
<br>
<h2><font size=6>WiDS Datathon 2024 - Challenge 2</font></h2>
<h3><font size=5>Modelos de regresión para estimación del periodo de diagnóstico metastático</font></h3>
<h3><font size=5>Parte 2 - Modelos de Regresión</font></h3>
<br>
<h1><font size=4>Alumna: Luna Jiménez Fernández</font></h1>
<br>



<div align="right">
<font size=3>Máster en Ciencia de Datos e Ingeniería de Datos en la Nube</font><br>
<font size=3>Universidad de Castilla-La Mancha</font>
</div>

<br>

---

INTRO A LA LIBRETA

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

# Array manipulation libraries
import numpy as np
import pandas as pd

# Regression models
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

# Importing visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

---

# Índice

* [3. Selección de atributos y preprocesamiento](#section3)
    * [3.1. Carga y particionamiento del conjunto de datos](#section3-1)
    * [3.2. Selección de atributos](#section3-2)
    * [3.3. Pre-procesamiento de los atributos seleccionados](#section3-3)
* [4. Selección de modelos de regresión](#section4)
* [5. Experimentación](#section5)
* [6. Análisis de resultados](#section6)
* [7. Conclusiones](#section7)
---

<a id="section3"></a>

# 3. Selección de atributos y preprocesamiento

Tras finalizar el **análisis exploratorio de datos** en la sección anterior, el siguiente paso en el proceso de ciencia de datos es el **preprocesamiento de la información** - para ser utilizada posteriormente por modelos de regresión, con el fin de predecir el tiempo de diagnóstico de la metástasis.

Concretamente, en este apartado se realizan las siguientes preparaciones:
- **Cargar y particionar** los conjuntos de datos en **entrenamiento**, **validación** y **test**.
- **Seleccionar el subconjunto de atributos** que van a ser utilizados durante la experimentación.
- **Preparar las *pipelines*** encargadas de transformar los datos crudos en datos listos para ser utilizados por los modelos posteriores.

<a id="section3-1"></a>

## 3.1. Carga y particionamiento del conjunto de datos

El conjunto de datos

In [74]:
# Loading the CSV files
df_train = pd.read_csv("data/train.csv", index_col="patient_id")
df_test = pd.read_csv("data/test.csv", index_col="patient_id")

# Display a small sample of both datasets to show that they have been properly loaded
display(df_train.sample(5))
display(df_test.sample(5))

,patient_race,payer_type,patient_state,patient_zip3,Region,Division,patient_age,patient_gender,bmi,breast_cancer_diagnosis_code,...,Average of Apr-18,Average of May-18,Average of Jun-18,Average of Jul-18,Average of Aug-18,Average of Sep-18,Average of Oct-18,Average of Nov-18,Average of Dec-18,metastatic_diagnosis_period
patient_id,,,,,,,,,,,,,,,,,,,,,
952915,NaN,COMMERCIAL,IL,604,Midwest,East North Central,45,F,NaN,1749,...,41.17,66.93,71.78,74.65,75.08,68.87,52.11,34.10,32.50,244
276728,Black,MEDICAID,CA,917,West,Pacific,44,F,NaN,C50412,...,66.06,65.90,72.14,80.81,79.66,75.22,70.54,65.23,58.38,259
523288,White,COMMERCIAL,AR,727,South,West South Central,60,F,28.29,C50211,...,49.86,71.73,77.09,79.06,75.82,71.58,59.84,41.76,40.15,28
668375,White,MEDICAID,OR,974,West,Pacific,65,F,33.13,C50411,...,51.38,58.26,61.21,67.76,66.56,60.52,55.01,47.61,43.20,72
796146,NaN,MEDICARE ADVANTAGE,PA,190,Northeast,Middle Atlantic,83,F,NaN,C50911,...,48.58,65.61,70.34,76.58,76.96,70.43,56.41,41.27,37.45,0


,patient_race,payer_type,patient_state,patient_zip3,Region,Division,patient_age,patient_gender,bmi,breast_cancer_diagnosis_code,...,Average of Mar-18,Average of Apr-18,Average of May-18,Average of Jun-18,Average of Jul-18,Average of Aug-18,Average of Sep-18,Average of Oct-18,Average of Nov-18,Average of Dec-18
patient_id,,,,,,,,,,,,,,,,,,,,,
433813,White,COMMERCIAL,OH,451,Midwest,East North Central,57,F,24.84,C50919,...,38.79,47.55,70.89,74.28,75.03,75.95,71.50,56.48,39.05,37.16
670302,Other,COMMERCIAL,NY,100,Northeast,Middle Atlantic,64,F,NaN,C50412,...,37.87,47.55,65.06,70.23,77.12,77.72,69.82,55.79,42.04,37.18
842199,Asian,COMMERCIAL,CA,920,West,Pacific,60,F,23.24,C50411,...,57.84,62.67,63.14,69.28,77.52,77.12,72.03,67.12,62.56,55.45
401700,White,MEDICAID,TN,385,South,East South Central,72,F,NaN,1749,...,45.99,50.74,71.03,74.90,76.23,75.28,73.83,59.59,43.05,41.63
917862,White,NaN,NM,882,West,Mountain,56,F,31.00,C50911,...,54.21,59.82,73.96,80.47,80.47,78.98,71.20,59.42,46.10,41.25


<a id="section3-2"></a>

## 3.2. Selección de atributos

<a id="section3-3"></a>

## 3.3. Pre-procesamiento de los atributos seleccionados

<a id="section4"></a>

# 4. Selección de modelos de regresión

<a id="section5"></a>

# 5. Experimentación

<a id="section6"></a>

# 6. Análisis de resultados